In [1]:
#!/usr/local/anaconda3/bin/python
import sys
import numpy as np
import utils as ut
from utils import tokenizer, check_category
import json
import re
from kfcd import kfcd_comp, FHNDLR, STRINP

In [2]:
#### MAIN ####

filename = './input_file/CrMn_fcc_afm_SWS_2.600.prn'#sys.argv[1]
with open(filename,'r') as f:
    lines = f.readlines()

# cleaning & get list type
clines, elines = ut.Cleaning(lines, opt='kfcd')

n_comp = kfcd_comp(elines)

values = list()
values.append(FHNDLR(clines))
values.append(STRINP(clines, n_comp))
